<a href="https://colab.research.google.com/github/MadmanMarble/MadmanMarble/blob/main/CCAP_UNET_processing_Fall2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import shutil
import hashlib

In [3]:
# # OR Unzipping to a specific directory
!mkdir /content/CCAP

In [81]:
!mkdir '/content/kahuku2022'
!mkdir '/content/kahuku2022/Images'
!mkdir '/content/kahuku2022/Labels'

In [82]:
# Unzip the file and place the files to the newly cerated directory (CCAP/image_chips/ISLAND)
!unzip /content/drive/MyDrive/data/wetland_project/processed_data/planet/kahuku2022.zip -d /content/CCAP/

Archive:  /content/drive/MyDrive/data/wetland_project/processed_data/planet/kahuku2022.zip
   creating: /content/CCAP/content/kahuku2022/Images/
  inflating: /content/CCAP/content/kahuku2022/Images/1_1.tif  
  inflating: /content/CCAP/content/kahuku2022/Images/1_1025.tif  
  inflating: /content/CCAP/content/kahuku2022/Images/1_1388.tif  
  inflating: /content/CCAP/content/kahuku2022/Images/1_513.tif  
  inflating: /content/CCAP/content/kahuku2022/Images/1025_1.tif  
  inflating: /content/CCAP/content/kahuku2022/Images/1025_1025.tif  
  inflating: /content/CCAP/content/kahuku2022/Images/1025_1388.tif  
  inflating: /content/CCAP/content/kahuku2022/Images/1025_513.tif  
  inflating: /content/CCAP/content/kahuku2022/Images/1537_1.tif  
  inflating: /content/CCAP/content/kahuku2022/Images/1537_1025.tif  
  inflating: /content/CCAP/content/kahuku2022/Images/1537_1388.tif  
  inflating: /content/CCAP/content/kahuku2022/Images/1537_513.tif  
  inflating: /content/CCAP/content/kahuku2022/Image

Next two blocks of code are probably unnecessary but run them for a sanity check.

In [83]:
# original images and labels folder: how many files are there?
org_img = '/content/CCAP/content/kahuku2022/Images'
org_img_files = os.listdir(org_img)
org_img_count = len(org_img_files)
print(f'The number of files in the folder is: {org_img_count}')
org_label = '/content/CCAP/content/kahuku2022/Labels'
org_label_files = os.listdir(org_label)
org_label_count = len(org_label_files)
print(f'The number of files in the folder is: {org_label_count}')

The number of files in the folder is: 24
The number of files in the folder is: 24


In [84]:
df = pd.read_csv('/content/CCAP/content/kahuku2022/kahuku2022.csv')

In [85]:
subset_df = df[(df['label'] > 12) & (df['label'] < 19)]

In [86]:
subset_df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,label,percent
30,2049,513,1537,1,4,606351.169499,607579.976243,2.400620e+06,2.401849e+06,15,0.002441
53,1025,1025,513,513,8,603893.584988,605122.386665,2.399392e+06,2.400620e+06,14,0.001041
54,1025,1025,513,513,8,603893.584988,605122.386665,2.399392e+06,2.400620e+06,15,0.018753
64,1537,1025,1025,513,9,605122.375646,606351.196371,2.399392e+06,2.400620e+06,13,0.001720
65,1537,1025,1025,513,9,605122.375646,606351.196371,2.399392e+06,2.400620e+06,14,0.000488
66,1537,1025,1025,513,9,605122.375646,606351.196371,2.399392e+06,2.400620e+06,15,0.077675
75,2049,1025,1537,513,10,606351.172433,607579.957629,2.399392e+06,2.400620e+06,14,0.015705
76,2049,1025,1537,513,10,606351.172433,607579.957629,2.399392e+06,2.400620e+06,15,0.081791
86,2561,1025,2049,513,11,607579.988327,608808.795540,2.399392e+06,2.400620e+06,15,0.001991
116,1537,1537,1025,1025,15,605122.383892,606351.190725,2.398163e+06,2.399392e+06,15,0.004395


In [87]:
subset_df = subset_df.copy()  # Create a copy of the subset DataFrame to avoid modifying the original
subset_df['tif_name'] = subset_df['x_org'].astype(str) + "_" + subset_df['y_org'].astype(str) + ".tif"

In [88]:
subset_df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,label,percent,tif_name
30,2049,513,1537,1,4,606351.169499,607579.976243,2.400620e+06,2.401849e+06,15,0.002441,1537_1.tif
53,1025,1025,513,513,8,603893.584988,605122.386665,2.399392e+06,2.400620e+06,14,0.001041,513_513.tif
54,1025,1025,513,513,8,603893.584988,605122.386665,2.399392e+06,2.400620e+06,15,0.018753,513_513.tif
64,1537,1025,1025,513,9,605122.375646,606351.196371,2.399392e+06,2.400620e+06,13,0.001720,1025_513.tif
65,1537,1025,1025,513,9,605122.375646,606351.196371,2.399392e+06,2.400620e+06,14,0.000488,1025_513.tif
66,1537,1025,1025,513,9,605122.375646,606351.196371,2.399392e+06,2.400620e+06,15,0.077675,1025_513.tif
75,2049,1025,1537,513,10,606351.172433,607579.957629,2.399392e+06,2.400620e+06,14,0.015705,1537_513.tif
76,2049,1025,1537,513,10,606351.172433,607579.957629,2.399392e+06,2.400620e+06,15,0.081791,1537_513.tif
86,2561,1025,2049,513,11,607579.988327,608808.795540,2.399392e+06,2.400620e+06,15,0.001991,2049_513.tif
116,1537,1537,1025,1025,15,605122.383892,606351.190725,2.398163e+06,2.399392e+06,15,0.004395,1025_1025.tif


In [89]:
# Copy the useable images files to new folder
source_images = '/content/CCAP/content/kahuku2022/Images'
destination_images = '/content/kahuku2022/Images'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_images, tif_name)
    destination_path = os.path.join(destination_images, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_images}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_images}")

Copied 1537_1.tif to /content/kahuku2022/Images
Copied 513_513.tif to /content/kahuku2022/Images
Copied 513_513.tif to /content/kahuku2022/Images
Copied 1025_513.tif to /content/kahuku2022/Images
Copied 1025_513.tif to /content/kahuku2022/Images
Copied 1025_513.tif to /content/kahuku2022/Images
Copied 1537_513.tif to /content/kahuku2022/Images
Copied 1537_513.tif to /content/kahuku2022/Images
Copied 2049_513.tif to /content/kahuku2022/Images
Copied 1025_1025.tif to /content/kahuku2022/Images
Copied 1537_1025.tif to /content/kahuku2022/Images
Copied 1537_1025.tif to /content/kahuku2022/Images
Copied 1537_1025.tif to /content/kahuku2022/Images
Copied 2049_1025.tif to /content/kahuku2022/Images
Copied 2049_1025.tif to /content/kahuku2022/Images
Copied 2274_1025.tif to /content/kahuku2022/Images
Copied 2274_1025.tif to /content/kahuku2022/Images
Copied 1537_1388.tif to /content/kahuku2022/Images
Copied 1537_1388.tif to /content/kahuku2022/Images
Copied 2049_1388.tif to /content/kahuku2022/

In [90]:
# Copy the useable labels files to new folder
source_labels = '/content/CCAP/content/kahuku2022/Labels'
destination_labels = '/content/kahuku2022/Labels'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_labels, tif_name)
    destination_path = os.path.join(destination_labels, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_labels}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_labels}")

Copied 1537_1.tif to /content/kahuku2022/Labels
Copied 513_513.tif to /content/kahuku2022/Labels
Copied 513_513.tif to /content/kahuku2022/Labels
Copied 1025_513.tif to /content/kahuku2022/Labels
Copied 1025_513.tif to /content/kahuku2022/Labels
Copied 1025_513.tif to /content/kahuku2022/Labels
Copied 1537_513.tif to /content/kahuku2022/Labels
Copied 1537_513.tif to /content/kahuku2022/Labels
Copied 2049_513.tif to /content/kahuku2022/Labels
Copied 1025_1025.tif to /content/kahuku2022/Labels
Copied 1537_1025.tif to /content/kahuku2022/Labels
Copied 1537_1025.tif to /content/kahuku2022/Labels
Copied 1537_1025.tif to /content/kahuku2022/Labels
Copied 2049_1025.tif to /content/kahuku2022/Labels
Copied 2049_1025.tif to /content/kahuku2022/Labels
Copied 2274_1025.tif to /content/kahuku2022/Labels
Copied 2274_1025.tif to /content/kahuku2022/Labels
Copied 1537_1388.tif to /content/kahuku2022/Labels
Copied 1537_1388.tif to /content/kahuku2022/Labels
Copied 2049_1388.tif to /content/kahuku2022/

In [91]:
# NEW images and labels folder: how many files are there?
new_img = '/content/kahuku2022/Images'
new_img_files = os.listdir(new_img)
new_img_count = len(new_img_files)
print(f'The number of files in the folder is: {new_img_count}')
new_label = '/content/kahuku2022/Labels'
new_label_files = os.listdir(new_label)
new_label_count = len(new_label_files)
print(f'The number of files in the folder is: {new_label_count}')

The number of files in the folder is: 12
The number of files in the folder is: 12


In [92]:
# Define the file path where you want to save the CSV file
csv_file_path = '/content/kahuku2022_subset.csv'
subset_df.to_csv(csv_file_path, index=False)
print(f"DataFrame exported to {csv_file_path}")

DataFrame exported to /content/kahuku2022_subset.csv


In [93]:
# Define the path to the 'wetland_categories' folder
folder_to_zip = '/content/kahuku2022'
zip_file_path = '/content/kahuku2022_subset.zip'
shutil.make_archive(zip_file_path[:-4], 'zip', folder_to_zip)
print(f'ZIP file created: {zip_file_path}')

ZIP file created: /content/kahuku2022_subset.zip


In [94]:
# Define the destination folder in Google Drive
drive_folder = '/content/drive/MyDrive/data/wetland_project/trainingdata'
shutil.move(zip_file_path, drive_folder)
print(f'ZIP file moved to Google Drive: {drive_folder}')
shutil.move(csv_file_path, drive_folder)
print(f'CSV file moved to Google Drive: {drive_folder}')

ZIP file moved to Google Drive: /content/drive/MyDrive/data/wetland_project/trainingdata
CSV file moved to Google Drive: /content/drive/MyDrive/data/wetland_project/trainingdata


In [95]:
stopppsps"(W)

SyntaxError: unterminated string literal (detected at line 1) (<ipython-input-95-871ba97a253f>, line 1)

# Define next folder

In [ ]:
# Unzip the file and place the files to the newly cerated directory (CCAP/image_chips/ISLAND)
!unzip /content/drive/MyDrive/data/wetland_project/processed_data/planet/Kahuku2021.zip -d /content/CCAP/

In [ ]:
!mkdir '/content/kahuku2021'
!mkdir '/content/kahuku2021/Images'
!mkdir '/content/kahuku2021/Labels'

In [ ]:
# original images and labels folder: how many files are there?
org_img = '/content/CCAP/content/kahuku2021/Images'
org_img_files = os.listdir(org_img)
org_img_count = len(org_img_files)
print(f'The number of files in the folder is: {org_img_count}')
org_label = '/content/CCAP/content/kahuku2021/Labels'
org_label_files = os.listdir(org_label)
org_label_count = len(org_label_files)
print(f'The number of files in the folder is: {org_label_count}')

In [ ]:
df = pd.read_csv('/content/CCAP/content/kahuku2021/kahuku2021.csv')

In [ ]:
df

In [ ]:
subset_df = df[(df['label'] > 12) & (df['label'] < 19)]

In [ ]:
subset_df = subset_df.copy()  # Create a copy of the subset DataFrame to avoid modifying the original
subset_df['tif_name'] = subset_df['x_org'].astype(str) + "_" + subset_df['y_org'].astype(str) + ".tif"

In [ ]:
subset_df

In [ ]:
# Copy the useable images files to new folder
source_images = '/content/CCAP/content/kahuku2021/Images'
destination_images = '/content/kahuku2021/Images'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_images, tif_name)
    destination_path = os.path.join(destination_images, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_images}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_images}")

In [ ]:
# Copy the useable labels files to new folder
source_labels = '/content/CCAP/content/kahuku2021/Labels'
destination_labels = '/content/kahuku2021/Labels'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_labels, tif_name)
    destination_path = os.path.join(destination_labels, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_labels}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_labels}")

In [ ]:
# NEW images and labels folder: how many files are there?
new_img = '/content/kahuku2021/Images'
new_img_files = os.listdir(new_img)
new_img_count = len(new_img_files)
print(f'The number of files in the folder is: {new_img_count}')
new_label = '/content/kahuku2021/Labels'
new_label_files = os.listdir(new_label)
new_label_count = len(new_label_files)
print(f'The number of files in the folder is: {new_label_count}')

In [ ]:
# Define the file path where you want to save the CSV file
csv_file_path = '/content/kahuku2021_subset.csv'
subset_df.to_csv(csv_file_path, index=False)
print(f"DataFrame exported to {csv_file_path}")

In [ ]:
# Define the path to the 'wetland_categories' folder
folder_to_zip = '/content/kahuku2021'
zip_file_path = '/content/kahuku2021_subset.zip'
shutil.make_archive(zip_file_path[:-4], 'zip', folder_to_zip)
print(f'ZIP file created: {zip_file_path}')

In [ ]:
# Define the destination folder in Google Drive
drive_folder = '/content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata'
shutil.move(zip_file_path, drive_folder)
print(f'ZIP file moved to Google Drive: {drive_folder}')
shutil.move(csv_file_path, drive_folder)
print(f'CSV file moved to Google Drive: {drive_folder}')

# one more to do 7, 8, 9 together

In [ ]:
# Unzip the file and place the files to the newly cerated directory (CCAP/image_chips/ISLAND)
!unzip /content/drive/MyDrive/wetland_unet/UNET_Image_Chips/Hawaii_Island_Update/Export/Hawaii_2005_009.zip -d /content/CCAP/

In [ ]:
# original images and labels folder: how many files are there?
org_img = '/content/CCAP/content/Hawaii_2005_009/Images'
org_img_files = os.listdir(org_img)
org_img_count = len(org_img_files)
print(f'The number of files in the folder is: {org_img_count}')
org_label = '/content/CCAP/content/Hawaii_2005_009/Labels'
org_label_files = os.listdir(org_label)
org_label_count = len(org_label_files)
print(f'The number of files in the folder is: {org_label_count}')

In [ ]:
df = pd.read_csv('/content/CCAP/content/Hawaii_2005_009/Hawaii_2005_009.csv')

In [ ]:
subset_df = df[(df['label'] > 12) & (df['label'] < 19)]

In [ ]:
subset_df = subset_df.copy()  # Create a copy of the subset DataFrame to avoid modifying the original
subset_df['tif_name'] = subset_df['x_org'].astype(str) + "_" + subset_df['y_org'].astype(str) + ".tif"

In [ ]:
subset_df

In [ ]:
# Copy the useable images files to new folder
source_images = '/content/CCAP/content/Hawaii_2005_009/Images'
destination_images = '/content/Hawaii_2005_009/Images'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_images, tif_name)
    destination_path = os.path.join(destination_images, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_images}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_images}")

In [ ]:
# Copy the useable labels files to new folder
source_labels = '/content/CCAP/content/Hawaii_2005_009/Labels'
destination_labels = '/content/Hawaii_2005_009/Labels'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_labels, tif_name)
    destination_path = os.path.join(destination_labels, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_labels}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_labels}")

In [ ]:
# NEW images and labels folder: how many files are there?
new_img = '/content/Hawaii_2005_009/Images'
new_img_files = os.listdir(new_img)
new_img_count = len(new_img_files)
print(f'The number of files in the folder is: {new_img_count}')
new_label = '/content/Hawaii_2005_009/Labels'
new_label_files = os.listdir(new_label)
new_label_count = len(new_label_files)
print(f'The number of files in the folder is: {new_label_count}')

In [ ]:
# Define the file path where you want to save the CSV file
csv_file_path = '/content/Hawaii_2005_009_subset.csv'
subset_df.to_csv(csv_file_path, index=False)
print(f"DataFrame exported to {csv_file_path}")

In [ ]:
# Define the path to the 'wetland_categories' folder
folder_to_zip = '/content/Hawaii_2005_009'
zip_file_path = '/content/Hawaii_2005_009_subset.zip'
shutil.make_archive(zip_file_path[:-4], 'zip', folder_to_zip)
print(f'ZIP file created: {zip_file_path}')

In [ ]:
# Define the destination folder in Google Drive
drive_folder = '/content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata'
shutil.move(zip_file_path, drive_folder)
print(f'ZIP file moved to Google Drive: {drive_folder}')
shutil.move(csv_file_path, drive_folder)
print(f'CSV file moved to Google Drive: {drive_folder}')